# Implent your own Decision Tree/Random Forest!


In this python notebook, you will create a basic decision tree on pandas data, and train a classifier on the Iris dataset. Then, you will implement a type of bagging and create a random forest classifier!

First, import the required modules:

In [41]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np

Then import and preview the data:

In [42]:
iris = load_iris()
df = pd.DataFrame(iris.data)
df['species'] = iris.target 
df.head()

,0,1,2,3,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


We have four features labeled 0, 1, 2, and 3. These stand for the length and the width of the sepals and petals, in centimeters. We want to use these four features to predict whether the species is one of three types of Iris plant, labeled 0, 1, or 2. 

Now, we split the dataset into training and test samples.

In [43]:
df['is_train'] = np.random.uniform(0, 1, len(df)) <= .75
train, test = df[df['is_train']==True], df[df['is_train']==False]
train = train.drop(['is_train'], axis = 1)
test = test.drop(['is_train'], axis = 1)

# Disorder (Splitting Metric)

First, we want to implement some measure of disorder in a set of data.

Implement either information gain or GINI impurity discussed in class. (for reference the equations are in 189 notes here https://www.eecs189.org/static/notes/n25.pdf) 


The argument `data` is a pandas dataframe containing the features and labels of several data points. We calculate disorder based on the labels, or the last column of the data. Note: make sure that you make this function work for different data (i.e. your function should work for data of different dimensions).

In [44]:
def disorder(data):
    sum = 0
    cat = [0, 1, 2, 3]
    n = data.shape[0]
    for i in cat:
        prob = ( data[data['species'].isin([i])].shape[0] ) / n
        sum += pow(prob, 2)
    return 1 - sum

We now create a split function. This function takes in a dataset, and indices for a row and column. We then return two dataframes split on the `column`th feature. The left dataset should contain all of the data where the `column`th feature is greater or equal to the `column`th feature of the `row`th datapoint, and the right should contain the rest. Use the disorder metric you implemented in the function above.

In [106]:
def split_on_row_column(data, row, column):
    left = data[data[column] >= data.loc[row, column]]
    right = data[data[column] < data.loc[row, column]]
    return left, right

# Find the best split by going through all the features and its rows.
# Select one with the lowest disorder (minimize entropy)
def find_best_split(data, cat):
    min_disorder = 100
    best_r, best_c = None, None
    for col in cat:
        for row in list(data.index.values):
            l, r = split_on_row_column(df, row, col)
            if l.shape[0] == 0:
                total_disorder = disorder(r)
            elif r.shape[0] == 0:
                total_disorder = disorder(l)
            else:
                total_disorder = disorder(l) + disorder(r)
            
            if total_disorder < min_disorder:
                min_disorder = total_disorder
                best_r = row
                best_c = col
    return best_r, best_c

We now want to define our recursive tree class. During training, there are two cases for a node. If the data is all one label, the node is a leaf node, and we return this value during inference. If the data is not all the same label, we find the best split of the data by iterating through all of features and rows in the data. Use the split function defined above to find the best split.

Inference takes in a row of a pandas dataframes and returns the predicted class. 

In [144]:
class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None
        self.split_r = None
        self.split_c = None
        self.isLeaf = False
    
    def train(self):
        if len(pd.Categorical(self.data["species"]).unique()) == 1:
            self.isLeaf = True
        else:
            columns = list(self.data.columns)
            self.split_r, self.split_c = find_best_split(self.data, columns[0:len(columns)-1])
            l, r = split_on_row_column(self.data, self.split_r, self.split_c)
            self.left = Node(l)
            self.right = Node(r)
            if (self.right.data.shape[0] == 0):
                self.left.isLeaf = True
            elif(self.left.data.shape[0] == 0):
                self.right.isLeaf = True
            else:
                self.left.train()
                self.right.train()
        pass
   
        
    def inference(self, x):
        if self.isLeaf:
            return pd.Categorical(self.data["species"]).unique()[0]
        val = x[self.split_c]
        key = self.data.loc[self.split_r, self.split_c]
        if val >= key:
            i = self.left.inference(x)
            return i
        elif val < key:
            i = self.right.inference(x)
            return i
        return -1

Now initialize and train a decision tree:

In [145]:
tree = Node(train)
tree.train()

Note that we don't check the training accuracy here (why?). We now want to validate our tree on the test dataset:

In [49]:
def validate(model, data):
    ct = 0
    corr = 0
    for i in range(test.shape[0]):
        data = test.iloc[i]
        ct += 1
        if model.inference(data) == data['species']:
            corr += 1
    return corr/ct

validate(tree, test)

0.9361702127659575

## Random Forest!

Now we will implement data bagging with a random forest! The set up is similar to a single tree. We pass in the data to the forest, along with hyperparameters `n`, `frac`, anbd `m`, which correspond to the number of trees, the fraction of the dataset to use in each bag, the number or percentage of random features (depending on your own implementation) selected at each possible split. Note that the difference between random forests and just bagging  is that random forests select a random subset of features per bag while bagging assumes all features are present in each sample. A good estimate for m in a dataset with `num_features` is m = sqrt(`num_features`). In the inference step we tally the number of votes from each decision tree and return the label with the most amount of votes.

In [151]:
class Forest:
    def __init__(self, data, n, frac, m):
        self.data = data
        self.n = n
        self.frac = frac
        self.m = m
    
    def train(self):
        self.trees = []
        num_cols = len(list(self.data.columns))
        for i in range(self.n):
            tree = Node(pd.merge(self.data.iloc[:, 0:4].sample(frac=self.frac).sample(n=int(self.m*num_cols),axis='columns'), 
                                 self.data.iloc[:, 4], left_index=True, right_index=True))
            tree.train()
            self.trees.append(tree)
    
    def inference(self, x):
        counter = {}
        for t in self.trees:
            infer = t.inference(x)
            if infer in counter:
                counter[infer] += 1
            else:
                counter[infer] = 1
        res = {key: val for key, val in sorted(counter.items(), key = lambda ele: ele[1], reverse = True)} 
        return list(res)[0]
#train.sample(frac= 0.5).sample(n=2, axis='columns').join(train.iloc[:, 4])
#list(train.index.values)

Train and validate your forest!

In [155]:
forest = Forest(train, 30, .5, 0.5)
forest.train()

In [156]:
validate(forest, test)

0.723404255319149